In [7]:
# 결정 트리, 확률적 경사 하강법은 테스트 세트를 반복해 사용하면서 체크를 한다.
# 그렇게 하다보면, 점점 성능을 테스트 세트에 맞춰서 과대적합이 되어버린다. 따라서 훈련,검증,테스트를 각각 60,20,20으로 나누어 진짜 판단할 때에만 검증 케이스를 사용하면 된다.
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)  # 훈련 80%, 테스트 20%
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42) # 훈련 80% 중 20%를 때어 약20%(16%이긴 함)을 검증 케이스로 사용한다.

print(sub_input.shape, val_input.shape) # (4157, 3) (1040, 3)

from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target)) # 0.99
print(dt.score(val_input, val_target)) # 0.86 훈련 세트에 과대적합 되어있음

# 교차검증 3,5,10-폴드 교차 검증 : 각각의 횟수만큼 훈련 케이스를 나눈 다음 각각의 일부분을 검증 케이스로 사용하고 평가해서 평균을 낸다. cross_validate(모델 객체, 훈련세트,훈련타깃) : 교차 검증 함수
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target) # 기본이 5-폴드 교차검증을 수행한다.
# fit time, score time, test score 딕셔너리를 반환한다. (훈련하는 시간, 검증하는 시간,테스트 점수)
print(scores)

import numpy as np
print(np.mean(scores['test_score'])) # 0.8553 scores의 test_score 열에 해당하는 내용 5개의 평균

from sklearn.model_selection import StratifiedKFold # 회귀모델 = KFold, 분류 모델 = StratifiedKFold 사용
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score'])) # 0.855

# n_splits = 10으로 10-폴드 교차검증 수행 가능
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt,train_input,train_target,cv=splitter)
print(np.mean(scores['test_score'])) # 0.857

# 하이퍼 파라미터 튜닝(max_depth,min_impurity_decrease 같은 매개변수 중 뭐가 가장 높은 점수를 받는지 여러 경우의 수로 돌리는 것이다.)
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]} # 1. 여러번 수행할 각각의 파라미터를 모아 배열 만들기
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1) # 2. GridSearchCV()로 (cv매개변수는 기본 5) 5x5개의 모델을 훈련 시키도록 gs에 전달
gs.fit(train_input,train_target) # 3. 훈련 세트 가져와서 훈련 시키기

dt = gs.best_estimator_ # best_estimator_ = 최적의 값을 적용한 모델
print(dt.score(train_input,train_target)) # 0.96 그러한 dt의 훈련점수
print(gs.best_params_) # {'min_impurity_decrease': 0.0001} 0.0001이 가장 좋은 값으로 선택됨.

print(gs.cv_results_['mean_test_score']) # [0.86819297 0.86453617 0.86492226 0.86780891 0.86761605] cv_results에 수행한 후의 결과가 나와있음. mean_test_score은 그 중에서 평균을 구해 놓은거임.
best_index = np.argmax(gs.cv_results_['mean_test_score']) # argmax() = 가장 큰 값의 인덱스 선택한다.
print(gs.cv_results_['params'][best_index]) # {'min_impurity_decrease': 0.0001}

# 여러개의 매개변수 중에서 가장 높은 점수 구하기
params = {'min_impurity_decrease':np.arange(0.0001,0.001,0.0001), # 어디까지는 불순도가 감소해야 끝남.
          'max_depth': range(5,20,1), # 최대 트리 자식 노드 개수
          'min_samples_split': range(2,100,10) # 노드를 나누기 위한 최소 샘플 수
          } # 1. 여러번 수행할 각각의 파라미터를 모아 배열 만들기
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1) # 2. GridSearchCV()로 (cv매개변수는 기본 5) 9x15x10 x 5 = 6750번 실행해서 최댓값을 찾는다.
gs.fit(train_input,train_target) # 3. 훈련 세트 가져와서 훈련 시키면서 최대인 값 구하기
print(gs.best_params_) # {'min_impurity_decrease': 0.0004, 'max_depth': 14, 'min_samples_split': 12}
print(np.max(gs.cv_results_['mean_test_score']))

# 랜덤 서치 : 값의 범위나 간격을 미리 정하기 어려울 때도 있고 너무 많은 매개변수 조건 때문에 수행 시간이 오래걸릴 수 있으므로 랜덤하게 골라서 쓴다.
from scipy.stats import uniform,randint
rgen = randint(0,10) # 0~10 정수 범위 설정
rgen.rvs(10) # 10개의 정수를 배열로 뽑기
np.unique(rgen.rvs(1000),return_counts=True) # 1000개 뽑기(중복된 값이 없도록 unique를 쓴다.) return_counts는 중복된 수가 몇번 반복됐는지 배열로 알려준다.
ugen = uniform(0,1) # 0~1 실수 범위 설정
ugen.rvs(10) # 10개의 실수를 배열로 뽑기

params = {'min_impurity_decrease':uniform(0.0001,0.001), # 어디까지는 불순도가 감소해야 끝남.
          'max_depth': randint(20,50), # 최대 트리 자식 노드 개수
          'min_samples_split': randint(4,20), # 노드를 나누기 위한 최소 샘플
          'min_samples_leaf': randint(3,20) # 리프 노드가 되기 위한 최소 샘플 수
          }
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42),params,n_iter=100,n_jobs=-1,random_state=42) # n_iter=100(100번 반복), n_jobs=-1(모든 cpu 사용)
gs.fit(train_input,train_target)
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))
dt = gs.best_estimator_ # 최적의 값을 적용한 모델
print(dt.score(train_input,train_target))

# 문제 3. splitter 매개변수는 기본적으로 best로 각 노드에서 최선의 분할을 찾는다. random이면 무작위로 분할한 뒤 가장 좋은 것을 고른다.
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random',random_state=42),params,n_iter=100,n_jobs=-1,random_state=42) # n_iter=100(100번 반복), n_jobs=-1(모든 cpu 사용)
gs.fit(train_input,train_target)
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))
dt = gs.best_estimator_ # 최적의 값을 적용한 모델
print(dt.score(train_input,train_target))

(4157, 3) (1040, 3)
0.9971133028626413
0.864423076923077
{'fit_time': array([0.0075326 , 0.00788379, 0.00757122, 0.00748992, 0.0072031 ]), 'score_time': array([0.00126696, 0.00145507, 0.00125837, 0.00130653, 0.00121379]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
0.855300214703487
0.855300214703487
0.8574181117533719
0.9615162593804117
{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]
{'min_impurity_decrease': 0.0001}
{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731
{'max_depth': 45, 'min_impurity_decrease': 0.0004046137691733707, 'min_samples_leaf': 4, 'min_samples_split': 7}
0.8699259643147996
0.8912834327496633
{'max_depth': 28, 'min_impurity_decrease': 0.0002759252526773454, 'min_samples_leaf': 6, 'min_samples_split': 4}
0.8493357148145406
0.8701173754088898
